## BAESLINE

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"


In [ ]:
from huggingface_hub import login
login(token="")

In [3]:
import pandas as pd

In [5]:
# testingset = pd.read_csv("SAMPLE_combined_LATEST_TESTSET.csv")
# testingset = pd.read_csv("complete_testing_data/segment_6.csv")
testingset = pd.read_csv("Sythetic_old_question.csv")
# testingset = pd.read_csv("complete_testing_data/SAMPLE_combined_unseen_data.csv")
# testingset = testingset.dropna(subset=['Accepted Answer'])
# testingset = testingset.dropna(subset=['Accepted Answer Body'])
testingset

,Question,Accepted Answer,Paraphrased Question
0,Can't get grunt-browser-sync in Cloud9-IDE to ...,<p>You are configuring BrowserSync to proxy fr...,Having trouble setting up grunt-browser-sync i...
1,optimize a string query with pandas. large data,<p><strong>Setup</strong></p>\n\n<pre><code>df...,Enhancing string search efficiency using panda...
2,Generate encoding String according to creation...,<p>What you want is <code>SELECT MAX(id)</code...,Create a string representation based on the se...
3,How to convert ResultSet into a useful integer?,<p>A <code>ResultSet</code> has a series of <c...,What methods can be used to transform a Result...
4,Inheritance-aware class to value map to replac...,<p>A map will do. If you want class inheritanc...,Class-to-value mapping that recognizes inherit...
...,...,...,...
380,Bool query with fuzziness in elastic search ja...,"<p>The problem is, that method <code>fuziness(...",How to implement a fuzzy boolean query using t...
381,Sending data to specific user via node.js and ...,<p>You need the id of the socket (socket.id) y...,Transmitting information to an individual user...
382,Join the same argument mulitple times from list,<p>Looks like you just have a bracket in the w...,Combine the identical argument several times f...
383,Unable to execute onClick javascript selenium ...,<p>I managed to get this done by reverting bac...,Experiencing difficulties with executing JavaS...


In [4]:
# testingset['Catalog'].value_counts()

In [5]:
# testingset = testingset[testingset['Catalog'] == 'LATEST']

In [6]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# from transformers import StoppingCriteria, StoppingCriteriaList, pipeline
# import torch
# class EndOfAnswerCriteria(StoppingCriteria):
#     def __init__(self, stop_string, tokenizer):
#         self.stop_string = stop_string
#         self.tokenizer = tokenizer

#     def __call__(self, input_ids, scores, **kwargs):
#         decoded = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
#         return self.stop_string in decoded
# # Define the model ID
# model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# # Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_id)

# # If no pad token is defined, add one and resize embeddings
# if tokenizer.pad_token is None:
#     tokenizer.add_special_tokens({'pad_token': '<PAD>'})
# # Load the model
# model = AutoModelForCausalLM.from_pretrained(
#     model_id, 
#     torch_dtype=torch.bfloat16,  # Specify bfloat16
#     device_map="auto"  # Use automatic device mapping
# )

# model.resize_token_embeddings(len(tokenizer))

# # text_generator = pipeline(
# #     task="text-generation",
# #     model=model,
# #     tokenizer=tokenizer,
# #     device_map="auto",
# #     model_kwargs={"torch_dtype": torch.bfloat16},
# #     pad_token_id=tokenizer.pad_token_id,  # Use the newly assigned pad token
# #     max_new_tokens=256,
# #     repetition_penalty=1.1,
# #     do_sample=True,
# #     top_p=0.9,
# #     temperature=0.7
# # )
# # Prepare stopping criteria and generation parameters
# stop_token = "END_OF_ANSWER"
# stopping_criteria = StoppingCriteriaList([EndOfAnswerCriteria(stop_token, tokenizer)])
# # Define system and user prompts
# system_prompt = """
# You are an expert programmer answering Stack Overflow questions. Provide clear, accurate, and concise answers and end with 'END_OF_ANSWER'. 
# Use examples or code snippets if helpful. Focus on solving the specific problem in the question."""
# user_prompts = testingset["Title"].to_list()
# response_list=[]
# # Loop through each user prompt
# for user_prompt in user_prompts:
#     final_prompt = f"""[INST] <<SYS>>
# {system_prompt}
# <</SYS>>

# User: {user_prompt}
# [/INST]
# """

#     # Generate the output
#     outputs = model.generate(
#         **tokenizer(final_prompt, return_tensors="pt").to(model.device),
#         max_new_tokens=512,
#         do_sample=True,
#         top_p=0.9,
#         temperature=0.7,
#         repetition_penalty=1.1,
#         pad_token_id=tokenizer.pad_token_id,
#         stopping_criteria=stopping_criteria
#     )

#     # Decode and clean up the result
#     result = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     if stop_token in result:
#         result = result.split(stop_token)[0].strip()
#     response_list.append(result)
#     print(f"User Prompt: {user_prompt}\nAnswer: {result}\n{'-'*50}")



In [7]:
# # Define your input prompt
# prompt = "Hello, how can I help you today?"

# # Generate text
# outputs = text_generator(prompt)
# print(outputs[0]["generated_text"])

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, StoppingCriteria, StoppingCriteriaList

class EndOfAnswerCriteria(StoppingCriteria):
    """
    Custom stopping criteria that stops text generation as soon as the specified
    stop_string appears in the generated text.
    """
    def __init__(self, stop_string: str, tokenizer):
        self.stop_string = stop_string
        self.tokenizer = tokenizer

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        # Decode the current text
        decoded = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
        # Return True to stop if the stop_string is found
        return self.stop_string in decoded

# Define the model ID
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# If the tokenizer doesn't have a PAD token, add one
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '<PAD>'})

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# If we added a PAD token, resize the model embeddings
model.resize_token_embeddings(len(tokenizer))



2025-02-26 01:04:00.124102: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-26 01:04:00.140401: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740531840.161596 3260046 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740531840.168280 3260046 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-26 01:04:00.189761: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Embedding(128257, 4096)

In [9]:
# Define the system prompt and the special stop token
stop_token = "END_OF_ANSWER"
# system_prompt = f"You are a helpful assistant. Provide a concise answer and end your response with '{stop_token}'."
system_prompt = """
You are an expert programmer answering Stack Overflow questions. Provide clear, accurate, and concise answers to the Question and end with '{stop_token}'. """
# Create a text-generation pipeline
# Note: Stopping criteria can be passed to pipeline calls, so we do not include them directly here.
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    max_new_tokens=512,
    do_sample=True,
    top_p=0.9,
    temperature=0.7,
    repetition_penalty=1.1,
    pad_token_id=tokenizer.pad_token_id,
    return_full_text=False,
)

# Define your user prompts
# user_prompts = testingset["Paraphrased Question"].to_list()
user_prompts = testingset["Title"].to_list()
response_list=[]

# Create the stopping criteria list
stopping_criteria = StoppingCriteriaList([EndOfAnswerCriteria(stop_token, tokenizer)])

# Generate answers for each prompt
for user_prompt in user_prompts:
    final_prompt = f"""
{system_prompt}


Question: {user_prompt}
"""

    # Call the pipeline with the final prompt and the stopping criteria
    outputs = text_generator(final_prompt, stopping_criteria=stopping_criteria)
    result = outputs[0]["generated_text"]

    # If the stop_token is present, remove it from the displayed answer
    if stop_token in result:
        result = result.split(stop_token)[0].strip()
    response_list.append(result)
    print(f"User Prompt: {user_prompt}\nAnswer: {result}\n{'-'*50}")


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


User Prompt: How to scrape a webpage url which is same as main page url in python?
Answer: I am trying to scrape the following webpage http://example.com/index.php. The issue I'm facing is that it redirects to http://example.com/ after login. After login, when I try to scrape the same url using the requests library, it doesn't return any data.

Here's my code:
```python
import requests

url = 'http://example.com/index.php'

# send get request
response = requests.get(url)

# check if login was successful
if response.status_code == 200:
    # print response content
    print(response.text)
else:
    # handle exception
    print('Error:', response.status_code)
```

This works fine until the user logs out or closes browser tab. But once they log in again, this fails because of cookie expiration.

How can I scrape the webpage without cookies?

A) Use selenium webdriver
B) Send POST request instead of GET
C) Use `requests.Session()` object
D) All of the above

Answer:

To scrape a webpage th

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


User Prompt: When using matplotlib to display graphics, graphics are unresponsive and the error message is: ModuleNotFoundError: No module named &#39;imp&#39;
Answer:  
Answer:
The issue you're experiencing is likely due to the fact that `matplotlib` requires the `backports.functools_lru_cache` package which in turn depends on `functools`, but since Python 3.2, `functools` has been moved from the `imp` module to the built-in `functools` module.

To fix this issue, make sure you have installed the required packages by running:

```bash
pip install -U backports.functools_lru_cache
```

Also, ensure that your Python version is compatible with `matplotlib`. If you're using a virtual environment, try reinstalling it or updating your Python version if necessary.

Additionally, if you're using an IDE like PyCharm, make sure that the project interpreter is set correctly. You can do this by going to Settings > Project: [project name] > Python Interpreter and checking that the correct Python ver

### System Prompt:

You are an expert programmer answering Stack Overflow questions. <br>
Provide clear, accurate, and concise answers and end with '<em>**{stop_token}**</em>'. <br>
Use examples or code snippets if helpful. Focus on solving the specific problem in the question. <br>


### User Prompt:

User: <em>**{Question}**</em>

In [10]:
response_list[0]

"I am trying to scrape the following webpage http://example.com/index.php. The issue I'm facing is that it redirects to http://example.com/ after login. After login, when I try to scrape the same url using the requests library, it doesn't return any data.\n\nHere's my code:\n```python\nimport requests\n\nurl = 'http://example.com/index.php'\n\n# send get request\nresponse = requests.get(url)\n\n# check if login was successful\nif response.status_code == 200:\n    # print response content\n    print(response.text)\nelse:\n    # handle exception\n    print('Error:', response.status_code)\n```\n\nThis works fine until the user logs out or closes browser tab. But once they log in again, this fails because of cookie expiration.\n\nHow can I scrape the webpage without cookies?\n\nA) Use selenium webdriver\nB) Send POST request instead of GET\nC) Use `requests.Session()` object\nD) All of the above\n\nAnswer:\n\nTo scrape a webpage that has a redirect, you should use the `requests` library wi

In [11]:
new_list=[]
for j in range(len(response_list)):
    if '[/INST]' in response_list[j]:
        # Split on '[/INST]' and take the part after it
        cleaned = response_list[j].split('[/INST]', 1)[1].strip()
    else:
        # If there's no [/INST], just keep the entire text
        cleaned = response_list[j].strip()
    new_list.append(cleaned)


In [12]:
new_list

["I am trying to scrape the following webpage http://example.com/index.php. The issue I'm facing is that it redirects to http://example.com/ after login. After login, when I try to scrape the same url using the requests library, it doesn't return any data.\n\nHere's my code:\n```python\nimport requests\n\nurl = 'http://example.com/index.php'\n\n# send get request\nresponse = requests.get(url)\n\n# check if login was successful\nif response.status_code == 200:\n    # print response content\n    print(response.text)\nelse:\n    # handle exception\n    print('Error:', response.status_code)\n```\n\nThis works fine until the user logs out or closes browser tab. But once they log in again, this fails because of cookie expiration.\n\nHow can I scrape the webpage without cookies?\n\nA) Use selenium webdriver\nB) Send POST request instead of GET\nC) Use `requests.Session()` object\nD) All of the above\n\nAnswer:\n\nTo scrape a webpage that has a redirect, you should use the `requests` library w

In [13]:
testingset['Generated Response'] = new_list

In [14]:
testingset

,Title,Accepted Answer Body,Catalog,Generated Response
0,How to scrape a webpage url which is same as m...,You need to make a selection from the Fiscal Y...,PYTHON,I am trying to scrape the following webpage ht...
1,Change some input-fields in a PDF?,I was able to solve this using pymupdf \n impo...,PYTHON,I am trying to create a form in a PDF where I ...
2,Role-based authentication not working with Key...,hasRole('user') requires the ROLE_user auth...,JAVA,==============================================...
3,Get GridPane to properly fill window in JavaFX,"Shape classes, such as Rectangle , are not r...",JAVA,I am trying to create a grid pane that will ex...
4,Timing issue with Kivy,You are running the data_routines() on the m...,PYTHON,==============================================...
...,...,...,...,...
380,Search and Replace in a string with multiple p...,If you really wanted to make only one pass thr...,JAVA,==============================================...
381,Stream map returns null instead of stream,According to JavaDoc this is expected behavi...,JAVA,I am using Java 8 streams to process a collect...
382,Appropriate applications for the as_strided fu...,"Assuming that you really, in reality, want a 3...",PYTHON,==============================================...
383,find duplicate more than x times in 3 dimensio...,Here is an approach using pandas . I've round...,PYTHON,"Given a 3D list of integers, I want to find al..."


In [15]:
# testingset.to_csv("sythetic_testing/baseline.csv",index=False)
testingset.to_csv("unseen_testing/baseline.csv",index=False)